In [1]:
import pandas as pd
import numpy as np

from Libreria.cargar_configbd import cargar_configbd
from Libreria.cargar_mes_a_procesar import cargar_mes_a_procesar

In [2]:
# Conectar a la base de datos PostgreSQL
conn = cargar_configbd.conectar_base_datos('conf_bd.txt')
# Crear un cursor para ejecutar consultas
cur = conn.cursor()

In [3]:
mes, año = cargar_mes_a_procesar.leer_csv_en_lista('mes_a_procesar.csv')
mes = 'mayo'
año = 2024
print(f"{mes}{año}")

mayo2024


In [4]:
# ------ DUPLICAR TABLA UNION VIAJES Y PUNTOS UDA
consulta_sql = f''' DROP TABLE IF EXISTS datos_uda_{mes}{año};
                    CREATE TABLE datos_uda_{mes}{año} AS
                        SELECT *
                        FROM join_viajes_puntos_{mes}{año};
                '''

# Ejecutar consulta
cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()

In [ ]:
# ------ ELIMINAR VIAJES FUERA DE HORAS DE CLASES
consulta_sql = f'''DELETE FROM datos_uda_{mes}{año}
                        WHERE NOT EXTRACT(DOW FROM recorded_at) BETWEEN 1 AND 5; -- 1: Lunes, 5: Viernes
                 DELETE FROM datos_uda_{mes}{año}
                        WHERE NOT EXTRACT(HOUR FROM recorded_at) BETWEEN 6 AND 20;
                 '''

'''cur.execute(consulta_sql)
# Confirma los cambios en la base de datos
conn.commit()'''

'cur.execute(consulta_sql)\n# Confirma los cambios en la base de datos\nconn.commit()'

In [ ]:
# ------ ELIMINAR  VIAJES QUE DUREN MENOS DE 2 MINUTOS, TENGAN MENOS DE 12 PTS Y QUE HAYAN RECORRIDO MENOS DE 1KM O MAS DE 30KM

def obtener_duracion_distancia_viaje(viaje):
    # Función para calcular la distancia usando la fórmula de Haversine
    def haversine(lat1, lon1, lat2, lon2):
        R = 6371  # Radio de la Tierra en kilómetros

        dlat = np.radians(lat2 - lat1)
        dlon = np.radians(lon2 - lon1)
        a = np.sin(dlat / 2) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * np.sin(dlon / 2) ** 2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
        distance = R * c  # Resultado en kilómetros
        return distance

    consulta_sql = f''' SELECT * from datos_uda_{mes}{año}
                        WHERE tripid = '{viaje[0]}' and uid = '{viaje[1]}' order by recorded_at;
                '''
    # Ejecutar consulta
    cur.execute(consulta_sql)
    rows = cur.fetchall()
    puntos_viaje = rows.copy()
    df_puntos_viaje = pd.DataFrame(puntos_viaje, columns=[desc[0] for desc in cur.description])

    # Calcular la distancia entre puntos consecutivos
    distances = []
    for i in range(1, len(df_puntos_viaje)):
        lat1, lon1 = df_puntos_viaje.iloc[i - 1][['latitude', 'longitude']]
        lat2, lon2 = df_puntos_viaje.iloc[i][['latitude', 'longitude']]
        distance = haversine(lat1, lon1, lat2, lon2)
        distances.append(distance)
    # Añadir una columna de distancia al DataFrame, con 0 en el primer punto
    df_puntos_viaje['distance'] = [0] + distances
    # Calcular la distancia total recorrida
    total_distance = sum(distances)

    # Calcular el tiempo total recorrido
    start_time = df_puntos_viaje['recorded_at'].iloc[0]
    end_time = df_puntos_viaje['recorded_at'].iloc[-1]
    total_time = end_time - start_time

    num_puntos = len(df_puntos_viaje)

    return total_time.total_seconds(), total_distance, num_puntos


consulta_sql = f''' SELECT distinct tripid, uid from datos_uda_{mes}{año};
                '''

# Ejecutar consulta
cur.execute(consulta_sql)
rows = cur.fetchall()
viajes = rows.copy()

datos_borrar = []
print(f"n_viajes: {len(viajes)}")
prog  =  0
for viaje in viajes:
    prog += 1
    print(f"Progreso: {prog}", end="\r")

    total_time, total_distance, num_puntos = obtener_duracion_distancia_viaje(viaje)
    # Verificar que sea un viaje representativo
    if (num_puntos) < 12 or (total_time <  120) or (total_distance <  1) or (total_distance > 30):
        datos_borrar.append(viaje)

df_borrar =  pd.DataFrame(datos_borrar,columns=['tripid', 'uid'])
# Guardar el DataFrame en un archivo CSV
df_borrar.to_csv(f'dataframe_viajes_borrar_{mes}{año}.csv', index=False)

n_viajes: 48576


In [5]:
df_borrar_p = pd.read_csv(f'dataframe_viajes_borrar_{mes}{año}.csv')

In [6]:
len(df_borrar_p)

17584

In [7]:
# Definir el tamaño del lote
batch_size = 1000

# Función para ejecutar un lote de DELETEs
def delete_batch(batch):
    trip_uid_pairs = ", ".join([f"('{row['tripid']}', '{row['uid']}')" for _, row in batch.iterrows()])
    consulta_delete = f'''
    DELETE FROM datos_uda_{mes}{año}
    WHERE (tripid, uid) IN ({trip_uid_pairs});
    '''
    cur.execute(consulta_delete)

# Ejecutar los DELETEs en lotes
print(f"Lotes: {len(df_borrar_p)/batch_size}")
prog = 0
for start in range(0, len(df_borrar_p), batch_size):
    prog += 1
    print(f"Progreso: {prog}", end="\r")

    batch = df_borrar_p.iloc[start:start + batch_size]
    delete_batch(batch)
    conn.commit()  # Hacer commit después de cada lote

# Cerrar la conexión
cur.close()
conn.close()

Lotes: 17.584
